# Simple InSAR baseline simulator

This notebook is a simple demonstration of Jupyter's interactive capabilities. We simulate a simple InSAR imaging scenario that includes:

1. Two SAR imaging platforms flying at altitudes of roughly 12-14 km, that can be repositioned by the user.
2. The platforms are imaging a swath that is 20-40km away to a side along the ground.
3. We simulate topography on the ground using simple points that can be rearranged by the user. 


We use simple 2D cartesian system to compute range differences and geometry characteristics to explain effect of InSAR baseline and topography on the observed fringe pattern.


In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/insar_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "insar_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "insar_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "insar_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

In [ ]:
# %matplotlib notebook
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.lines import Line2D
import numpy as np
from scipy.interpolate import interp1d

###Sets up basic point on the plot that the user can drag and play with
class DraggablePoint:

    # http://stackoverflow.com/questions/21654008/matplotlib-drag-overlapping-points-interactively

    lock = None #  only one can be animated at a time

    def __init__(self, parent, x=0.1, y=0.1, size=500, color='r'):

        self.parent = parent
        self.point = patches.Ellipse((x, y), size, size, fc=color, alpha=0.5, edgecolor=color)
        self.x = x
        self.y = y
        parent.fig.axes[0].add_patch(self.point)
        self.press = None
        self.background = None
        self.connect()

    def connect(self):

        'connect to all the events we need'

        self.cidpress = self.point.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.cidrelease = self.point.figure.canvas.mpl_connect('button_release_event', self.on_release)
        self.cidmotion = self.point.figure.canvas.mpl_connect('motion_notify_event', self.on_motion)


    def on_press(self, event):

        if event.inaxes != self.point.axes:
            return
        
        if DraggablePoint.lock is not None:
            return
        
        contains, attrd = self.point.contains(event)
        if not contains:
            return
        
        self.press = (self.point.center), event.xdata, event.ydata
        DraggablePoint.lock = self

        # draw everything but the selected rectangle and store the pixel buffer
        canvas = self.point.figure.canvas
        axes = self.point.axes
        self.point.set_animated(True)

        canvas.draw()
        self.background = canvas.copy_from_bbox(self.point.axes.bbox)

        # now redraw just the rectangle
        axes.draw_artist(self.point)

        # and blit just the redrawn area
        canvas.blit(axes.bbox)
        #canvas.draw()


    def on_motion(self, event):

        if DraggablePoint.lock is not self:
            return
        if event.inaxes != self.point.axes: 
            return
        self.point.center, xpress, ypress = self.press
        dx = event.xdata - xpress
        dy = event.ydata - ypress
        self.point.center = (self.point.center[0]+dx, self.point.center[1]+dy)

        canvas = self.point.figure.canvas
        axes = self.point.axes
        # restore the background region
        canvas.restore_region(self.background)

        # redraw just the current rectangle
        axes.draw_artist(self.point)

        self.x = self.point.center[0]
        self.y = self.point.center[1]

        # blit just the redrawn area
        canvas.blit(axes.bbox)
        #canvas.draw()


    def on_release(self, event):

        'on release we reset the press data'
        if DraggablePoint.lock is not self:
            return

        self.press = None
        DraggablePoint.lock = None

        # turn off the rect animation property and reset the background
        self.point.set_animated(False)

        self.background = None

        # redraw the full figure
        self.point.figure.canvas.draw()

        self.x = self.point.center[0]
        self.y = self.point.center[1]
        
        self.parent.updateFigure()

    def disconnect(self):

        'disconnect all the stored connection ids'

        self.point.figure.canvas.mpl_disconnect(self.cidpress)
        self.point.figure.canvas.mpl_disconnect(self.cidrelease)
        self.point.figure.canvas.mpl_disconnect(self.cidmotion)

In [ ]:
##Simple class that simulates the imaging geometry and updates plots.
class InSARGeometry:
    '''
    Class to simulate an InSAR imaging scenario
    '''
    
    def __init__(self, wvl=0.24, width=10, height=6):
        '''
        Create figure and populate points on it.
        '''
        #Save wavelength
        self.wvl = wvl
        
        #Create figure
        self.fig = plt.figure(figsize=(width,height))
        
        #Create axes to include imaging targets and SAR positions
        self.imgrect = [0.1, 0.4, 0.8, 0.5]
        self.imgaxes = self.fig.add_axes(self.imgrect)
        
        #Create axes to show actual range difference (unwrapped)
        self.unwrect = [0.1, 0.1, 0.8, 0.1]
        self.unwaxes = self.fig.add_axes(self.unwrect)
        
        #Create axes to show wrapped range difference (fringes)
        self.ifgrect = [0.1, 0.25, 0.8, 0.1]
        self.ifgaxes = self.fig.add_axes(self.ifgrect)
 
        #x-axis multiplier
        self.xscale = 40000.
        
        #Y-axis multiplier
        self.yscale = 15000.

        self.imgaxes.set_xlim([0, self.xscale])
        self.imgaxes.set_ylim([0, self.yscale])
        self.bpartext = []
        self.bperptext = []

        plt.show()
        

        
        #Add target points on ground
        self.targets = []
        self.addTargets()
        
        #Add SAR antennas
        self.SARs = []
        self.addSARs()
        
        #Add UnwPlot
        self.unwplot = None
        self.ifgimage = None
        self.addUnwPlot()
        self.imgaxes.set_ylabel('Y')
        
        self.updateFigure()
        
    def position2Canvas(self, x, y):
        '''
        Convert physical x,y to canvas position for plotting
        '''
        xpos = self.imgrect[0] + (x/self.xscale) * self.imgrect[2]
        ypos = self.imgrect[1] + (y/self.yscale) * self.imgrect[3]
        return xpos, ypos
    
    def canvas2Position(self, xpos, ypos):
        '''
        Convert canvas position to physical x,y
        '''
        x = (xpos - self.imgrect[0]) * (self.xscale / self.imgrect[2])
        y = (ypos - self.imgrect[1]) * (self.yscale / self.imgrect[3])
        return x, y
        
    def addTargets(self):
        '''
        We will add about 8 targets covering the right half of the X-axis
        '''
        
        xloc = np.linspace(0.5,0.9, num=16) * self.xscale
        for val in xloc:
            xp, yp = self.position2Canvas(val, 500.)
            self.targets.append( DraggablePoint(self, val, 500.))
            
            
    def addSARs(self):
        '''
        We will add 2 SAR platforms at top left
        '''
        self.SARs.append( DraggablePoint(self, 1000., 14000., color='b', size=1000))
        self.SARs.append( DraggablePoint(self, 3000., 13000., color='g', size=1000))
        
    def getBaseline(self, center):
        '''
        Compute baseline w.r.t given position
        '''
        baseline = [self.SARs[1].x - self.SARs[0].x,
                    self.SARs[1].y - self.SARs[0].y]
        look = [center[0] - self.SARs[0].x,
                center[1] - self.SARs[0].y]
        
        lookvec = np.array(look) / np.linalg.norm(look)
        Bpar = np.dot(baseline, lookvec)
        Bperp = np.sqrt(np.linalg.norm(baseline)**2 - Bpar*Bpar)
        Bsign = np.sign( np.cross(look+[0.], baseline+[0.])[2])
        Bperp *= Bsign

        return Bpar, Bperp
    
    def addUnwPlot(self, update=False):
        '''
        Create unw plot
        '''
        xpts = []
        ypts = []
        
        for pt in self.targets:
            xpts.append(pt.x)
            ypts.append(pt.y)
        
        xpts = np.array(xpts)
        ypts = np.array(ypts)
        
        ind = np.argsort(xpts)
        xpts = xpts[ind]
        ypts = ypts[ind]
        
        intp = interp1d(xpts, ypts, assume_sorted=True, bounds_error=False)
        
        xgrid = np.linspace(0, 1.0, num=100) * self.xscale
        ygrid = intp(xgrid)
        
        rng1 = np.sqrt((xgrid- self.SARs[0].x)**2 + (ygrid - self.SARs[0].y)**2)
        rng2 = np.sqrt((xgrid - self.SARs[1].x)**2 + (ygrid - self.SARs[1].y)**2)
        unw = rng1 - rng2
        phs1 = 4 * np.pi * unw/ self.wvl
        phs1 = phs1 - 2*np.pi*np.round(phs1/(2*np.pi))
        
        phs = np.zeros((10, unw.size))
        phs[:,:] = phs1[None,:]
        
        bparnear, bperpnear = self.getBaseline([xpts[0], ypts[0]])
        bparfar, bperpfar = self.getBaseline([xpts[-1], ypts[-1]])
        
        if not update:
            self.unwplot, = self.unwaxes.plot(xgrid, unw)
            self.unwaxes.set_xlim([0., self.xscale])
            self.unwaxes.set_ylabel('$\Delta r$')
            self.unwaxes.set_xlabel('X')
            self.ifgimage = self.ifgaxes.imshow(phs, clim=[-np.pi,np.pi], 
                                                extent=[0,self.xscale, 0, self.yscale],
                                                cmap=plt.get_cmap('hsv'),
                                                aspect='auto')
            self.ifgaxes.set_ylabel("$\\frac{4 \pi}{\lambda} \cdot \Delta r$")
            self.bperptext = self.imgaxes.text(0.65*self.xscale, 14000, 
                                               'Bperp={0:.2f}, {1:.2f}'.format(bperpnear, bperpfar))
            self.bpartext = self.imgaxes.text(0.65*self.xscale, 12000, 
                                              'Bpar={0:.2f}, {1:.2f}'.format(bparnear, bparfar))
            #self.ifgplot.imshow(phs, clim=[0, 2*np.pi], cmap='hsv')
        else:
            self.unwplot.set_ydata(unw)
            self.unwplot.set_xdata(xgrid)
            self.ifgimage.set_data(phs)
            self.bperptext.set_text('Bperp={0:.2f}, {1:.2f}'.format(bperpnear, bperpfar))
            self.bpartext.set_text('Bpar={0:.2f}, {1:.2f}'.format(bparnear, bparfar))
        
        self.unwaxes.set_ylim([np.nanmin(unw)-10, np.nanmax(unw)+10])
            
    
    
    def updateFigure(self):
        '''
        Update the whole plot
        '''
        self.addUnwPlot(update=True)
        self.fig.canvas.draw()
        

In [ ]:
insar = InSARGeometry(wvl=100.)

One is encouraged to interact with the plot and perform following experiments:


1. The plots themselves display Bparallel and Bperp at near and far range - notice the difference as you reposition the platforms. 

2. Align the two platforms as best as possible to the center off swath (30km line) and notice the fringe pattern reduce and Bperp pattern decrease.

3. Drag individual points in the swath and rearrange them to form topographic features and observe the fringe pattern change. 